In [1]:
import cv2
import numpy as np

def compute_background_avg(video_path, num_frames=30, resize_dim=(512, 512), apply_blur=True):
    """
    Computes the average background image from the first 'num_frames' frames of a video.
    """
    cap = cv2.VideoCapture(video_path)
    count = 0
    avg_frame = None

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    num_frames = min(num_frames, total_frames)

    while count < num_frames:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.resize(frame, resize_dim)

        if apply_blur:
            frame = cv2.GaussianBlur(frame, (5, 5), 0)

        if avg_frame is None:
            avg_frame = np.float32(frame)
        else:
            cv2.accumulateWeighted(frame, avg_frame, 0.05)

        count += 1

    cap.release()
    return cv2.convertScaleAbs(avg_frame)

def background_subtraction(video_path, background_img, resize_dim=(512, 512), threshold=30, apply_blur=True):
    """
    Performs background subtraction using the computed average background.
    """
    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, resize_dim)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if apply_blur:
            gray_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)

        background_resized = cv2.resize(background_img, resize_dim)
        if apply_blur:
            background_resized = cv2.GaussianBlur(background_resized, (5, 5), 0)

        diff = cv2.absdiff(gray_frame, background_resized)
        _, fg_mask = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

        # Noise removal: Opening followed by Closing
        kernel = np.ones((3, 3), np.uint8)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)

        # Show both original and mask
        stacked = np.hstack((gray_frame, fg_mask))
        cv2.imshow("Original | Foreground", stacked)

        if cv2.waitKey(30) & 0xFF == 27:  # ESC key to stop
            break

    cap.release()
    cv2.destroyAllWindows()

# Paths to indoor and outdoor video files
indoor_video_path = "indoor.mp4"
outdoor_video_path = "outdoor.mp4"

# Resize dimension
resize_dim = (512, 512)

# Compute background images
print("Computing Indoor Background...")
indoor_background = compute_background_avg(indoor_video_path, resize_dim=resize_dim)

print("Computing Outdoor Background...")
outdoor_background = compute_background_avg(outdoor_video_path, resize_dim=resize_dim)

# Perform background subtraction
print("Processing Indoor Video...")
background_subtraction(indoor_video_path, indoor_background, resize_dim=resize_dim)

print("Processing Outdoor Video...")
background_subtraction(outdoor_video_path, outdoor_background, resize_dim=resize_dim)


Computing Indoor Background...
Computing Outdoor Background...
Processing Indoor Video...
Processing Outdoor Video...


In [2]:
import cv2
import numpy as np
import random

# ========== Settings ==========
video_path = "op.mp4"  # Replace with your actual video
save_output = True             # Set to False if you don't want to save the output
min_area = 1500
min_width, min_height = 30, 30
alpha = 0.03                   # For background averaging
frame_size = (512, 512)        # Resize all frames

# ========== Helper for Colored IDs ==========
colors = {}
def get_color(object_id):
    if object_id not in colors:
        colors[object_id] = (random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))
    return colors[object_id]

# ========== Start Processing ==========
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
frame = cv2.resize(frame, frame_size)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
background = gray.astype("float")

# Optional: video writer
if save_output:
    out = cv2.VideoWriter("tracked_output.avi", cv2.VideoWriter_fourcc(*"XVID"), 20, frame_size)

object_id = 0
centroids_prev = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, frame_size)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.accumulateWeighted(gray, background, alpha)
    bg = cv2.convertScaleAbs(background)

    # Background Subtraction
    diff = cv2.absdiff(gray, bg)
    _, fg_mask = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY)

    # Morphological operations
    kernel = np.ones((3, 3), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.dilate(fg_mask, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    centroids_curr = {}
    for cnt in contours:
        if cv2.contourArea(cnt) < min_area:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        if w < min_width or h < min_height:
            continue

        centroid = (int(x + w / 2), int(y + h / 2))

        # Assign object ID (basic nearest-centroid tracking)
        matched_id = None
        for cid, prev_centroid in centroids_prev.items():
            if abs(prev_centroid[0] - centroid[0]) < 30 and abs(prev_centroid[1] - centroid[1]) < 30:
                matched_id = cid
                break

        if matched_id is None:
            matched_id = object_id
            object_id += 1

        centroids_curr[matched_id] = centroid

        color = get_color(matched_id)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f"ID {matched_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.circle(frame, centroid, 4, color, -1)

    centroids_prev = centroids_curr.copy()

    # Display
    cv2.imshow("Tracked Objects", frame)
    cv2.imshow("Foreground", fg_mask)

    if save_output:
        out.write(frame)

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
if save_output:
    out.release()
cv2.destroyAllWindows()


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
